In [2]:
%load_ext autoreload
%autoreload 2
from get_data import *
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


In [3]:
train_data_15m = get_data(
    "2021-11-01",
    "2021-12-31",
    "BTCUSDT",
    "15m"
)

train_data_1h = get_data(
    "2021-10-01",
    "2021-12-31",
    "BTCUSDT",
    "1h"
)

train_data_12h = get_data(
    "2021-01-01",
    "2021-12-31",
    "BTCUSDT",
    "12h"
)

train_data_1d = get_data(
    "2020-01-01",
    "2021-12-31",
    "BTCUSDT",
    "1d"
)

  1%|          | 34/5760 [00:00<00:17, 330.10it/s]

Preparing data from database


  2%|▏         | 35/2184 [00:00<00:06, 341.24it/s]

Preparing data from database


 10%|▉         | 70/730 [00:00<00:01, 344.47it/s]

Preparing data from database


100%|██████████| 730/730 [00:02<00:00, 327.09it/s]


In [4]:
def grid_search(
    forecaster,
    data,
    col,
    ohlcv_size,
):
    # Lags used as predictors
    lags_grid = [6, 12, 18]

    # Regressor's hyperparameters
    param_grid = {
        'n_estimators': [100, 250, 500],
        'max_depth': [4, 6, 10]
    }

    print(f"RESULT FOR column = {col} with size = {ohlcv_size} with regressor {forecaster}")
    return grid_search_forecaster(
        forecaster=forecaster,
        y=data[col],
        param_grid=param_grid,
        lags_grid=lags_grid,
        steps=1,
        refit=False,
        metric='mean_squared_error',
        initial_train_size=int(len(data)*0.5),
        fixed_train_size=False,
        return_best=True,
        verbose=False
    )

In [7]:
# HYPER PARAMETER TUNINNG CATBOOST 1h low
result = grid_search(
    forecaster=ForecasterAutoreg(regressor=CatBoostRegressor(random_state=123, silent=True), lags=1),
    data=train_data_1h,
    col="low",
    ohlcv_size="1h"
)
result

loop lags_grid:   0%|                                               | 0/3 [00:00<?, ?it/s]

RESULT FOR column = low with size = 1h with regressor ================= 
ForecasterAutoreg 
Regressor: <catboost.core.CatBoostRegressor object at 0x00000200DD14BBE0> 
Lags: [1] 
Window size: 1 
Included exogenous: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: {'loss_function': 'RMSE', 'silent': True, 'random_state': 123} 
Creation date: 2022-04-10 23:22:52 
Last fit date: None 
Skforecast version: 0.4.3 

Number of models compared: 27


loop lags_grid: 100%|███████████████████████████████████████| 3/3 [01:42<00:00, 34.14s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4 5 6] 
  Parameters: {'max_depth': 6, 'n_estimators': 500}
  Backtesting metric: 449687.0502730848



,lags,params,metric,max_depth,n_estimators
5,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 500}",4.496871e+05,6,500
4,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 250}",4.504990e+05,6,250
8,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 500}",4.550332e+05,10,500
7,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 250}",4.874440e+05,10,250
2,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 500}",5.043405e+05,4,500
6,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 100}",5.240509e+05,10,100
1,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 250}",5.287987e+05,4,250
3,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 100}",5.419821e+05,6,100
0,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 100}",5.876649e+05,4,100
10,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 4, 'n_estimators': 250}",6.948968e+05,4,250


In [5]:
# HYPER PARAMETER TUNINNG CATBOOST 15m low
result = grid_search(
    forecaster=ForecasterAutoreg(regressor=XGBRegressor(random_state=123), lags=1),
    data=train_data_1h,
    col="low",
    ohlcv_size="1h"
)
result

loop lags_grid:   0%|                                               | 0/3 [00:00<?, ?it/s]

RESULT FOR column = low with size = 1h with regressor ================= 
ForecasterAutoreg 
Regressor: XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             random_state=123, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None) 
Lags: [1] 
Window size: 1 
Included exogenous: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: {'objective': 'reg:squarederror', 'base_score': No

loop lags_grid: 100%|███████████████████████████████████████| 3/3 [01:08<00:00, 22.94s/it]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4 5 6] 
  Parameters: {'max_depth': 6, 'n_estimators': 100}
  Backtesting metric: 416822.1254405018



,lags,params,metric,max_depth,n_estimators
3,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 100}",4.168221e+05,6,100
6,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 100}",4.201379e+05,10,100
7,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 250}",4.213880e+05,10,250
8,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 500}",4.213891e+05,10,500
4,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 250}",4.265448e+05,6,250
5,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 500}",4.270939e+05,6,500
0,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 100}",4.521859e+05,4,100
1,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 250}",4.774959e+05,4,250
2,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 500}",4.812804e+05,4,500
9,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 4, 'n_estimators': 100}",5.070679e+05,4,100


In [6]:
# HYPER PARAMETER TUNINNG CATBOOST 15m low
result = grid_search(
    forecaster=ForecasterAutoreg(regressor=LGBMRegressor(random_state=123), lags=1),
    data=train_data_1h,
    col="low",
    ohlcv_size="1h"
)
result

loop lags_grid:   0%|                                               | 0/3 [00:00<?, ?it/s]

RESULT FOR column = low with size = 1h with regressor ================= 
ForecasterAutoreg 
Regressor: LGBMRegressor(random_state=123) 
Lags: [1] 
Window size: 1 
Included exogenous: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': 123, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0} 
Creation date: 2022-04-10 22:58:36 
Last fit date: None 
Skforecast version: 0.4.3 

Number of models compared: 27


loop lags_grid: 100%|███████████████████████████████████████| 3/3 [00:15<00:00,  5.29s/it]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4 5 6] 
  Parameters: {'max_depth': 4, 'n_estimators': 100}
  Backtesting metric: 369991.7238474248



,lags,params,metric,max_depth,n_estimators
0,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 100}",369991.723847,4,100
1,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 250}",379709.320752,4,250
3,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 100}",381978.616753,6,100
6,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 100}",388920.919593,10,100
2,"[1, 2, 3, 4, 5, 6]","{'max_depth': 4, 'n_estimators': 500}",390416.647754,4,500
4,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 250}",393711.440870,6,250
7,"[1, 2, 3, 4, 5, 6]","{'max_depth': 10, 'n_estimators': 250}",405127.120582,10,250
5,"[1, 2, 3, 4, 5, 6]","{'max_depth': 6, 'n_estimators': 500}",406966.142391,6,500
15,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 10, 'n_estimators': 100}",407397.281868,10,100
10,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","{'max_depth': 4, 'n_estimators': 250}",407885.680124,4,250
